# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [49]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
res_hyg_score = list(establishments.find(query))
# Use count_documents to display the number of documents in the result
print(f'The number of documents is: ',establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(res_hyg_score[0])

The number of documents is:  41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6645aeef2275192a0b123853'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}

In [54]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f'The number of rows in the DataFrame is : ',len(df))
# Display the first 10 rows of the DataFrame
print(df.head(10))

The number of rows in the DataFrame is :  41
                        _id   FHRSID  ChangesByServerID  \
0  6645aeef2275192a0b123853   110681                  0   
1  6645aeef2275192a0b123bd3   612039                  0   
2  6645aef02275192a0b123edd   730933                  0   
3  6645aef02275192a0b1240cc   172735                  0   
4  6645aef02275192a0b1240da   172953                  0   
5  6645aef02275192a0b124a7a   512854                  0   
6  6645aef02275192a0b124c9b  1537089                  0   
7  6645aef02275192a0b1261c6   155648                  0   
8  6645aef02275192a0b12660a  1012883                  0   
9  6645aef02275192a0b126e1c   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [35]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName' : 'London',
         'RatingValue': {'$gte' :4}
        }

# Use count_documents to display the number of documents in the result
results = establishments.count_documents(query)
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
doc_results = establishments.find(query)
pprint(doc_results)

In [55]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(doc_results)
# Display the number of rows in the DataFrame
print('Number of rows: ', len(df))
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows:  0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [63]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 50.769705
longitude = 0.27694

query = {
    'geocode.latitude': {'$gte': str(latitude - degree_search), '$lte': str(latitude + degree_search)},
    'geocode.longitude': {'$gte': str(longitude - degree_search), '$lte': str(longitude + degree_search)},
    'RatingValue': 5
}
projection = {
    '_id': 0,
    'BusinessName': 1,
    'RatingValue': 1,
    'HygieneScore': 1,
    'geocode.latitude': 1,
    'geocode.longitude': 1
}

sort = [('HygieneScore', 1)]
limit = 10
cursor = establishments.find(query, projection).sort(sort).limit(limit)
search_results = list(cursor)
# Print the results
pprint(search_results)

[{'BusinessName': 'Da Vinci Hotel',
  'RatingValue': 5,
  'geocode': {'latitude': '50.763522', 'longitude': '0.286765'}},
 {'BusinessName': 'Citrus Hotel Eastbourne',
  'RatingValue': 5,
  'geocode': {'latitude': '50.761449', 'longitude': '0.284707'}},
 {'BusinessName': 'Howard Square Kiosk',
  'RatingValue': 5,
  'geocode': {'latitude': '50.7627601623535', 'longitude': '0.28642800450325'}},
 {'BusinessName': 'Best Western Lansdowne Hotel',
  'RatingValue': 5,
  'geocode': {'latitude': '50.761023', 'longitude': '0.284417'}},
 {'BusinessName': 'Oban Hotel',
  'RatingValue': 5,
  'geocode': {'latitude': '50.76237', 'longitude': '0.285901'}},
 {'BusinessName': 'Carpet Gardens Kiosk',
  'RatingValue': 5,
  'geocode': {'latitude': '50.7627601623535', 'longitude': '0.28642800450325'}},
 {'BusinessName': 'Wish Tower Kiosk',
  'RatingValue': 5,
  'geocode': {'latitude': '50.760872', 'longitude': '0.286154'}},
 {'BusinessName': 'Airstream Mobile Catering Unit',
  'RatingValue': 5,
  'geocode': 

In [64]:
# Convert result to Pandas DataFrame
search_df = pd.DataFrame(search_results)
search_df

,BusinessName,RatingValue,geocode
0,Da Vinci Hotel,5,"{'longitude': '0.286765', 'latitude': '50.7635..."
1,Citrus Hotel Eastbourne,5,"{'longitude': '0.284707', 'latitude': '50.7614..."
2,Howard Square Kiosk,5,"{'longitude': '0.28642800450325', 'latitude': ..."
3,Best Western Lansdowne Hotel,5,"{'longitude': '0.284417', 'latitude': '50.7610..."
4,Oban Hotel,5,"{'longitude': '0.285901', 'latitude': '50.76237'}"
5,Carpet Gardens Kiosk,5,"{'longitude': '0.28642800450325', 'latitude': ..."
6,Wish Tower Kiosk,5,"{'longitude': '0.286154', 'latitude': '50.7608..."
7,Airstream Mobile Catering Unit,5,"{'longitude': '0.286154', 'latitude': '50.7608..."
8,Bistrot Pierre,5,"{'longitude': '0.286154', 'latitude': '50.7608..."
9,The View Hotel,5,"{'longitude': '0.286676', 'latitude': '50.7631..."


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [24]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match' : {'scores.Hygiene' : {'$eq' : 0}}}
# 2. Groups the matches by Local Authority
group_query = {'$group' : {'_id' : '$LocalAuthorityName', 'count' : {'$sum' : 1}}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort' : {'count' : -1}}

#pipeline 
pipeline = [match_query, group_query, sort_values]
# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results)

[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542},
 {'_id': 'Colchester', 'count': 498},
 {'_id': 'Tunbridge Wells', 'count': 491},
 {'_id': 'Folkestone and Hythe', 'count': 480},
 {'_id': 'Eastbourne', 'count': 478},
 {'_id': 'Hastings', 'count': 464},
 {'_id': 'Bromley', 'count': 460},
 {'_id': 'Ashford', 'count': 427},
 {'_id': 'Havering', 'count': 397},
 {'_id': 'Dartford', 'count': 383},
 {'_id': 'Braintree', 'count': 382},
 {'_id': 'Basildon', 'count': 362},
 {'_id': 'Gravesham', 'count': 339},
 {'_id': 'Tonbridge and Malling', 'count': 325},
 {'_id': 'Thurrock', 'count': 313},
 {'_id': 'Barking and Dagenham', 'count': 301},
 {'_id': 'Canterbury City', 'count': 298},
 {'_id':

In [29]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print('The number of rows is: ', len(results_df))
# Display the first 10 rows of the DataFrame
df.head(10)

The number of rows is:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
